In [2]:
import polars as pl
import numpy as np

In [9]:
df = pl.scan_parquet('./Datasets/crsp')
df.head().collect_schema()

Schema([('PERMNO', Int64),
        ('date', String),
        ('NAMEENDT', String),
        ('SHRCD', Int64),
        ('EXCHCD', Int64),
        ('NCUSIP', String),
        ('TICKER', String),
        ('COMNAM', String),
        ('SHRCLS', String),
        ('PRIMEXCH', String),
        ('TRDSTAT', String),
        ('SECSTAT', String),
        ('PERMCO', Int64),
        ('CUSIP', String),
        ('DCLRDT', String),
        ('DLAMT', Float64),
        ('DLPDT', String),
        ('DLSTCD', Float64),
        ('PAYDT', String),
        ('RCRDDT', String),
        ('SHRFLG', Float64),
        ('DISTCD', Float64),
        ('DIVAMT', Float64),
        ('FACPR', Float64),
        ('FACSHR', Float64),
        ('ACPERM', Float64),
        ('ACCOMP', Float64),
        ('SHRENDDT', String),
        ('NWPERM', Float64),
        ('DLRETX', String),
        ('DLPRC', Float64),
        ('DLRET', String),
        ('BIDLO', Float64),
        ('ASKHI', Float64),
        ('PRC', Float64),
        ('VOL', F

In [ ]:
# efficient daily log returns for Apple
aapl_permno = df.filter(pl.col('TICKER') == 'AAPL') \
    .select('PERMNO') \
    .unique() \
    .collect() \
    .item()

log_returns = df.filter(pl.col('PERMNO') == aapl_permno) \
    .select(['PERMNO', 'PRC']) \
    .with_columns(
        (pl.col('PRC')/pl.col('PRC').shift(1)) \
            .log()
            .alias('LOG_RET')
        ) \
    .collect()

PERMNO,PRC,LOG_RET
i64,f64,f64
14593,21.28,null
14593,22.17,0.040972
14593,22.09,-0.003615
14593,22.59,0.022382
14593,23.36,0.033518
…,…,…
14593,258.20001,0.011413
14593,259.01999,0.003171
14593,255.59,-0.013331


In [4]:
# compustat rolling sum
df_cstat = pl.scan_parquet('./Datasets/compustat/fundamentals_quarterly_all')
print(df_cstat.collect_schema())

Schema({'costat': String, 'curcdq': String, 'datafmt': String, 'indfmt': String, 'consol': String, 'gvkey': String, 'datadate': Date, 'conm': String, 'tic': String, 'cusip': String, 'cik': String, 'exchg': Int32, 'fyr': Int16, 'fic': String, 'add1': String, 'add2': String, 'add3': String, 'add4': String, 'addzip': String, 'busdesc': String, 'city': String, 'conml': String, 'county': String, 'dldte': Date, 'dlrsn': String, 'ein': String, 'fax': String, 'fyrc': Int16, 'ggroup': String, 'gind': String, 'gsector': String, 'gsubind': String, 'idbflag': String, 'incorp': String, 'ipodate': Date, 'loc': String, 'naics': String, 'phone': String, 'prican': String, 'prirow': String, 'priusa': String, 'sic': String, 'spcindcd': Int32, 'spcseccd': Int32, 'spcsrc': String, 'state': String, 'stko': Int16, 'weburl': String, 'acctchgq': String, 'acctstdq': String, 'adrrq': Decimal(precision=18, scale=4), 'ajexq': Decimal(precision=24, scale=12), 'ajpq': Decimal(precision=24, scale=12), 'apdedateq': Da

In [5]:
df_ibes = pl.scan_parquet("./Datasets/ibes/summary_stats")
df_ibes.collect_schema()

Schema([('ticker', String),
        ('cusip', String),
        ('statpers', Date),
        ('fpedats', Date),
        ('fpi', String),
        ('meanest', Float64),
        ('medest', Float64),
        ('stdev', Float64),
        ('numest', Float64),
        ('highest', Float64),
        ('lowest', Float64),
        ('actual', Float64),
        ('anndats_act', Date),
        ('usfirm', Int64)])

In [6]:
df_om = pl.scan_parquet('./Datasets/optionmetrics/options/all_prices')
df_om.collect_schema()

Schema([('secid', Float64),
        ('date', Datetime(time_unit='ns', time_zone=None)),
        ('exdate', Datetime(time_unit='ns', time_zone=None)),
        ('cp_flag', Categorical),
        ('strike_price', Float32),
        ('best_bid', Float32),
        ('best_offer', Float32),
        ('volume', Float64),
        ('open_interest', Float64),
        ('impl_volatility', Float32),
        ('delta', Float32),
        ('gamma', Float32),
        ('vega', Float32),
        ('theta', Float32),
        ('contract_size', Float32),
        ('forward_price', Float32),
        ('__index_level_0__', Int64)])

In [ ]:
df_om_sf = pl.scan_parquet('./Datasets/optionmetrics/options/all_surface')
df_om_sf.collect_schema()

Schema([('secid', Float64),
        ('date', Datetime(time_unit='ns', time_zone=None)),
        ('days', Int16),
        ('delta', Float32),
        ('impl_volatility', Float32),
        ('dispersion', Float32),
        ('cp_flag', Categorical),
        ('__index_level_0__', Int64)])

In [8]:
df_om_sd = pl.scan_parquet('./Datasets/optionmetrics/options/all_standardized')
df_om_sd.collect_schema()

Schema([('secid', Float64),
        ('date', Datetime(time_unit='ns', time_zone=None)),
        ('days', Int16),
        ('cp_flag', Categorical),
        ('impl_volatility', Float32),
        ('delta', Float32),
        ('gamma', Float32),
        ('vega', Float32),
        ('theta', Float32),
        ('premium', Float32),
        ('forward_price', Float32),
        ('__index_level_0__', Int64)])